In [ ]:
import joblib
import pandas as pd
import numpy as np


# Opdele Test sæt i corona og ikke corona

In [ ]:
Q1_corona = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_base.pkl")
Q2_corona = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_base.pkl")
Q4_corona = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_base.pkl")

Q1_corona_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_train_base.pkl")
Q2_corona_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_train_base.pkl")
Q4_corona_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_train_base.pkl")

In [ ]:
from pandas.tseries.offsets import DateOffset
def end_date_function (df,h):
    end_dates = pd.to_datetime(df["slutmåned"])+DateOffset(months=h)
    return end_dates


In [ ]:
Q1_corona["end_date"] = Q1_corona.apply(lambda x: end_date_function(df=x,h=3),axis=1)
Q2_corona["end_date"] = Q2_corona.apply(lambda x: end_date_function(df=x,h=6),axis=1)
Q4_corona["end_date"] = Q4_corona.apply(lambda x: end_date_function(df=x,h=12),axis=1)



In [ ]:
Q1_corona["corona"] = Q1_corona.end_date.apply(lambda x: "Corona" if x.year >2019 else "Corona_negative")
Q2_corona["corona"] = Q2_corona.end_date.apply(lambda x: "Corona" if x.year >2019 else "Corona_negative")
Q4_corona["corona"] = Q4_corona.end_date.apply(lambda x: "Corona" if x.year >2019 else "Corona_negative")


In [ ]:
Q1_corona.corona.value_counts(normalize=True).sort_index()

In [ ]:
Q2_corona.corona.value_counts(normalize=True).sort_index()

In [ ]:
Q4_corona.corona.value_counts(normalize=True).sort_index()

In [ ]:
Q1_corona[Q1_corona.end_date.dt.year==2018].corona.value_counts()

In [ ]:
Q4_corona

In [ ]:
def metric_dataframe (corona, test_df):
  if corona =="true":
    test_df = test_df[test_df.corona=="Corona"]
  elif corona=="false":
    test_df = test_df[test_df.corona=="Corona_negative"]
  else: 
    raise ValueError("Der er en fejl i koden")

  return test_df


# Mase function
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np

  # Metric Calculations
def metric_calculations (metric_test_df,metric_train_df):
  # Mae
  analytiker_mae = mean_absolute_error(metric_test_df["EPS_actual"],metric_test_df["analyst_EPS_mean"])
  model_mae = mean_absolute_error(metric_test_df["EPS_actual"],metric_test_df["Model_predicted"])

  # Outperformance MAE

  outperformance_mae = -(model_mae-analytiker_mae)

  # Outperformance %

  outperformance_pct = -(model_mae/analytiker_mae-1)


  metric_df = pd.DataFrame.from_dict({"Outperformance MAE":outperformance_mae,"Outperformance MAE(%)":outperformance_pct},orient="index")
  return metric_df

In [ ]:
! pip install rpy2
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages("forecast")
forecast_r = importr("forecast")



In [ ]:

def p_value (generel_df):
  generel_df["model_error"] =generel_df["Model_predicted"]- generel_df["EPS_actual"]
  generel_df["analyst_error"] =generel_df["analyst_EPS_mean"]- generel_df["EPS_actual"]



  with localconverter(ro.default_converter + pandas2ri.converter):
    r_temp = ro.conversion.py2rpy(generel_df)

  p_value_ = forecast_r.dm_test(r_temp.rx2["analyst_error"],r_temp.rx2["model_error"],alternative="greater",power=1).rx2["p.value"]
  p_value_ = np.asarray(p_value_,dtype=np.float32)

  return p_value_

In [ ]:
Corona_liste = ["true","false"]
train_df_files =[Q1_corona_train,Q2_corona_train,Q4_corona_train]
test_df_files =[Q1_corona,Q2_corona,Q4_corona]
performance_df_true = {}
performance_df_false = {}
p_values_true=[]
p_values_false=[]

list_of_dict =[]
for boolean in Corona_liste:
  for element in zip(zip(train_df_files,test_df_files),["Q1","Q2","Q4"]):
    test_df_temp = metric_dataframe(corona=f'{boolean}',test_df=element[0][1])
    if boolean =="true":
      performance_df_true[f'{element[1]}']=metric_calculations(metric_test_df=test_df_temp,metric_train_df = element[0][0])
      p_values_true.append((p_value(test_df_temp)))
    else:
      performance_df_false[f'{element[1]}']=metric_calculations(metric_test_df=test_df_temp,metric_train_df = element[0][0])
      p_values_false.append((p_value(test_df_temp)))


p_values_df_samlet = pd.DataFrame([np.asarray(p_values_true).flatten(),np.asarray(p_values_false).flatten()])
p_values_df_samlet.index=["corona","ikke corona"]
p_values_df_samlet.columns = ["Q1","Q2","Q4"]

df_true = pd.concat(performance_df_true)
df_true["corona"] = "true"

df_false = pd.concat(performance_df_false)
df_false["corona"] = "false"
samlet_performance = pd.concat([df_true,df_false]).reset_index().set_index(["corona","level_0","level_1"])

    

In [ ]:
samlet_performance.round(4)

In [ ]:
p_values_df_samlet

In [ ]:
joblib.dump(samlet_performance,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/robusthed/performance_corona.pkl")
joblib.dump(p_values_df_samlet,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/robusthed/p_values_corona.pkl")

# Andre Trænings perioder

In [ ]:
! pip install catboost
import numpy as np
import pandas as pd

## Funktioner

In [ ]:
def ml_pipeline (fil,train_start):
  # importing df and basic cleaning
  df_ml = pd.read_csv(fil)
  df_ml["Forecasttidspunkt"] = pd.to_datetime(df_ml["Forecasttidspunkt"])
  df_ml["periode_regnskabstal"] = pd.to_datetime(df_ml["periode_regnskabstal"])
  df_ml.set_index("Forecasttidspunkt",inplace=True)


  #Dropping non relevant columns
  df_ml.rename(columns={"CUSIP":"cusip"},inplace=True)
  list_drop = ["SHROUT","company_name","cusip","analyst_EPS_mean","analyst_high","analyst_low","analyst_std_mean","periode_regnskabstal","PERMNO","date","slutmåned","Instrument"]
  df_ml.drop(list_drop,axis=1,inplace=True,errors="ignore")

  # Creating year and month
  df_ml["month"] = df_ml.index.month
  df_ml["year"] = df_ml.index.year
  df_ml = df_ml.astype({'GVKEY': 'category'})
  df_ml = df_ml.astype({'industry_fama': 'category'})
  df_ml.sort_index(inplace=True)
  
  # New features og duplikat kolonner
  df_ml["volume_usd"] = df_ml["ALTPRC"] * df_ml["VOL"]
  df_ml["Div_yield"] = df_ml["DPS_ex_date(dvpsxq)"] / df_ml["ALTPRC"]
  df_ml.drop(["PRC","adj_close"],axis=1,inplace=True)

  columns_drop_list = ["Assets_total(Atq)","industry_return","VOL","volume_usd"]
  df_ml.drop(columns_drop_list,axis=1,inplace=True)


  # Lagging variable

  columns_lag = ['EPS_actual', 'Accounts_payable(Apq)',
       'Assets_total(Atq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'GVKEY', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)','PRC',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
    'VOL', 'adj_return','adj_close','market_cap', 'niq', 'industry_return', 'merafkast', 'ALTPRC','volume_usd','Div_yield']

  df_lag = df_ml.copy()
  df_lag = df_lag[df_lag.columns.intersection(columns_lag)]
  df_lag_new = df_lag.copy()

  for column in df_lag.columns:
    for i in range(1,6):
      df_lag_new[f'{column}_{i}'] = df_lag.groupby("GVKEY")[f'{column}'].shift(i).values

  columns_to_choose = ['Assets_total(Atq)','Accounts_payable(Apq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
       'market_cap', 'niq',"Div_yield","ALTPRC",'adj_return',"industry_return","merafkast",'VOL','volume_usd','PRC','adj_close']
  col_lag_choose = []
  for element in columns_to_choose:
    for i in range(1,6):
      col_lag_choose.append(f'{element}_{i}')
  col_lag_choose.append("GVKEY")

  # Vælger kun de endelige features
  df_lag_new = df_lag_new[df_lag_new.columns.intersection(col_lag_choose)]

  # Merge lags og oprindelig dataframe sammen
  df_ml = df_ml.reset_index().merge(df_lag_new.reset_index(),how="inner",on=["GVKEY","Forecasttidspunkt"])

  # Fikse indeks og erstatte NA i lags med NA

  numeric_columns = df_ml.select_dtypes(include=['number']).columns
  df_ml[numeric_columns] = df_ml[numeric_columns].fillna(0)

  df_ml.set_index("Forecasttidspunkt",inplace=True)
  

  cutoff_date = "2016-06-30"
  validation_start = "2016-09-30"
  validation_end = "2018-06-30"
  test_start = "2018-09-30"

  training_df = df_ml.loc[:cutoff_date,:]
  validation_df = df_ml.loc[validation_start:validation_end,:]
  test_df = df_ml.loc[test_start:,:]

  training_df_cv =  df_ml.loc[train_start:validation_end,:]

# X og Y

  training_y = training_df["EPS_actual"]
  validatation_y = validation_df["EPS_actual"]
  test_y = test_df["EPS_actual"]
  training_y_cv = training_df_cv["EPS_actual"]

  training_x = training_df.drop("EPS_actual",axis=1)
  training_x_cv = training_df_cv.drop("EPS_actual",axis=1)
  validatation_x = validation_df.drop("EPS_actual",axis=1)
  test_x = test_df.drop("EPS_actual",axis=1)


  from catboost import CatBoostRegressor
  from catboost import Pool

  params = {'boosting_type': 'Plain','bootstrap_type': 'MVS',
 'colsample_bylevel': 0.09413483344915186,'depth': 8,
 'iterations': 1000,
 "loss_function": "MAE",
  "max_ctr_complexity": 0,
  "one_hot_max_size": 5000,
  "cat_features": ["GVKEY","industry_fama"]}

  cat_model = CatBoostRegressor(**params,random_seed=2021)


  cat_model.fit(training_x_cv,training_y_cv,verbose=False)

  print(cat_model.best_iteration_)
  print(cat_model.get_best_score())

  test_pred = cat_model.predict(test_x)

  return test_pred

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np


  # Metric Dataframe
def metric_dataframe (test_predicted, fil="/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input.csv.zip"):

  '''fil : Indsæt stien til den fil der indeholder data før ML rensning
    test_predicted : Henvis til et array af predicted værdier på testsættet'''

  df = pd.read_csv(fil)
  df["Forecasttidspunkt"] = pd.to_datetime(df["Forecasttidspunkt"])
  df.set_index("Forecasttidspunkt",inplace=True)
  df.sort_index(inplace=True)
  træning_slut = "2018-06-30"
  test_start = "2018-09-30"
  train_df = df.loc[:træning_slut,:]
  test_df = df.loc[test_start:,:]
  test_df["Model_predicted"] = test_predicted
  test_df.dropna(subset=["analyst_EPS_mean"],inplace=True)

 
  return test_df

  # Metric Calculations
def metric_calculations (metric_test_df,base_pkl):
  # Mae
  model_mae = mean_absolute_error(metric_test_df["EPS_actual"],metric_test_df["Model_predicted"])
  baseline_mae = joblib.load(base_pkl)["MAE"]["Model"]
  print(model_mae,baseline_mae)

  # Outperformance MAE

  outperformance_mae = -(model_mae-baseline_mae)

  # Outperformance %

  outperformance_pct = -(model_mae/baseline_mae-1)


  metric_df = pd.DataFrame.from_dict({"Outperformance MAE":outperformance_mae,"Outperformance MAE(%)":outperformance_pct},orient="index")
  return metric_df

In [ ]:
joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_performance_base.pkl")["MAE"]["Model"]

In [ ]:
joblib.load(pkl_fil)["MAE"]["Model"]

In [ ]:
start_datoer = ["2003-06-30","2005-06-30", "2007-06-30", "2009-06-30", "2011-06-30","2013-06-30","2015-06-30"]

fil_sti = "/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input.csv.zip"
pkl_fil = "/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_performance_base.pkl"

Q1_performance = {}

Q1_test = {}

for dato in start_datoer:
  Q1_test[f'{dato}'] = metric_dataframe(test_predicted=(ml_pipeline(fil=fil_sti,train_start=f'{dato}')),fil=fil_sti)
  Q1_performance[f'{dato}'] = metric_calculations((Q1_test[f'{dato}']),base_pkl = pkl_fil)




In [ ]:
pd.concat(Q1_performance)

### Q2

In [ ]:
start_datoer = ["2003-06-30","2005-06-30", "2007-06-30", "2009-06-30", "2011-06-30","2013-06-30","2015-06-30"]

fil_sti = "/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input_2Q.csv.zip"
pkl_fil = "/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_performance_base.pkl"

Q2_performance = {}

Q2_test = {}

for dato in start_datoer:
  Q2_test[f'{dato}'] = metric_dataframe(test_predicted=(ml_pipeline(fil=fil_sti,train_start=f'{dato}')),fil=fil_sti)
  Q2_performance[f'{dato}'] = metric_calculations((Q2_test[f'{dato}']),base_pkl=pkl_fil)

In [ ]:
pd.concat(Q2_performance)

### Q4

In [ ]:
start_datoer = ["2003-06-30","2005-06-30", "2007-06-30", "2009-06-30", "2011-06-30","2013-06-30","2015-06-30"]

fil_sti = "/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input_4Q.csv.zip"
pkl_fil = "/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_performance_base.pkl"

Q4_performance = {}

Q4_test = {}

for dato in start_datoer:
  Q4_test[f'{dato}'] = metric_dataframe(test_predicted=(ml_pipeline(fil=fil_sti,train_start=f'{dato}')),fil=fil_sti)
  Q4_performance[f'{dato}'] = metric_calculations((Q4_test[f'{dato}']),base_pkl=pkl_fil)

In [ ]:
Q4_performance

In [ ]:
joblib.dump(Q1_performance,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/robusthed/Q1_performance_trainchange")
joblib.dump(Q2_performance,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/robusthed/Q2_performance_trainchange")
joblib.dump(Q4_performance,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/robusthed/Q4_performance_trainchange")

In [ ]:
q1_perf_samlet = pd.concat(Q1_performance)
q2_perf_samlet = pd.concat(Q2_performance)
q4_perf_samlet = pd.concat(Q4_performance)

q1_perf_samlet["horizon"] = 1
q2_perf_samlet["horizon"] = 2
q4_perf_samlet["horizon"] = 4

In [ ]:
df_samlet_perf = pd.concat([q1_perf_samlet,q2_perf_samlet,q4_perf_samlet])

plot_df = df_samlet_perf.reset_index()[df_samlet_perf.reset_index().level_1=="Outperformance MAE(%)"]

plot_df["level_0"] = pd.to_datetime(plot_df["level_0"])

In [ ]:
plot_df

In [ ]:
cutoff.year

In [ ]:
liste_ = []

cutoff = pd.Timestamp("2018-06-30")

plot_df["year"] = plot_df.level_0.apply(lambda x: cutoff.year - x.year)
plot_df.set_index("year",inplace=True)
plot_df.sort_index(ascending=False,inplace=True)


In [ ]:
plot_df

In [ ]:
import matplotlib.pyplot as plt

for horizon in plot_df.horizon.unique():
  temp_df = plot_df[plot_df.horizon ==horizon]
  plt.plot(temp_df[0],label = f'forecast horizon {horizon}')
  plt.ylim(-0.04,0.04)
  
plt.legend()
axis = plt.gca()
axis.invert_xaxis()

plt.show()

In [ ]:
plot_df.horizon.unique()